In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable
from torch.autograd import grad as torch_grad
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
from statistics import mean as stat_mean

In [2]:
# Load Data
data = pd.read_csv('fulldata.csv', index_col = 'Date')
data.shape

(2492, 50)

In [3]:
# Visualize Dataset
%matplotlib notebook

# Create Subplot Figure for all Columns
subplotFig, subplotAx = plt.subplots(3,2, num='Basic Indicator History ($AAPL)')

# Plot All Data Columns (Open,High,Low,Close,& Volume)
data['open_aapl'].plot(ax=subplotAx[0,0])
data['high_aapl'].plot(ax=subplotAx[0,1])
data['low_aapl'].plot(ax=subplotAx[1,0])
data['close_aapl'].plot(ax=subplotAx[1,1])
data['volume_aapl'].plot(ax=subplotAx[2,0])

# Adjust Figure Layout
plt.tight_layout(pad = 1.5)

# Add Subplot Titles
subplotAx[0,0].set_title("Open")
subplotAx[0,1].set_title("High")
subplotAx[1,0].set_title("Low")
subplotAx[1,1].set_title("Close")
subplotAx[2,0].set_title("Volume")

<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Volume')

In [4]:
# Read over ARIMA predictions
%store -r full_arima_predictions

# Increment Indices
arima_predictions_list = full_arima_predictions.tolist()
arima_predictions_list.append(arima_predictions_list.pop(0))

# Append ARIMA predictions to dataframe
data['arima'] = arima_predictions_list

# Append close to dataframe
data['y'] = data['close_aapl']

In [5]:
#Data preview
data

,open_aapl,high_aapl,low_aapl,close_aapl,volume_aapl,open_googl,high_googl,low_googl,close_googl,volume_googl,...,momentum,rsi_5,rsi_15,rsi_ratio,tr,atr,roc,vader_polarity,arima,y
Date,,,,,,,,,,,,,,,,,,,,,
5/9/2013,16.421786,16.535714,16.270714,16.313214,398749904,21.792098,22.012812,21.726785,21.808114,87945080,...,15.313214,66.171526,64.804242,1.021099,0.295000,0.419158,0.165081,0.131248,16.313212,16.313214
5/10/2013,16.356071,16.418214,16.088572,16.177500,334849928,21.903956,22.034833,21.825130,22.027075,75955060,...,15.177500,57.572386,61.822287,0.931256,0.329643,0.412934,0.159885,0.146785,16.191360,16.177500
5/13/2013,16.125357,16.353572,16.125000,16.240714,317109240,21.993543,22.083127,21.855684,21.959510,57893080,...,15.240714,60.556778,62.679325,0.966136,0.228572,0.404487,0.140643,0.063889,16.231099,16.240714
5/14/2013,16.208928,16.257143,15.791071,15.852143,447115760,21.958759,22.238780,21.949751,22.198992,63190600,...,14.852143,39.310535,54.606048,0.719893,0.466071,0.379742,0.092901,0.049072,15.893159,15.852143
5/15/2013,15.684286,15.750000,15.084286,15.316071,741612360,22.409198,22.931701,22.372159,22.919439,159658020,...,14.316071,24.492002,45.872465,0.533915,0.767857,0.412962,0.057682,0.011088,15.359893,15.316071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3/27/2023,159.940000,160.770000,157.870000,158.280000,52390266,104.615000,104.760000,101.927300,102.460000,31120864,...,157.280000,58.221301,60.577495,0.961104,2.900000,3.498250,0.028928,0.154446,158.428987,158.280000
3/28/2023,157.970000,158.490000,155.980000,157.650000,45992152,102.440000,102.450000,99.740000,101.030000,32057865,...,156.650000,53.185052,59.113390,0.899712,2.510000,3.543964,0.039908,0.171291,157.627915,157.650000
3/29/2023,159.370000,161.050000,159.350000,160.770000,51305691,102.280000,102.490000,100.650000,101.390000,28779572,...,159.770000,69.511345,63.760854,1.090188,3.400000,3.478964,0.051678,0.149073,160.714492,160.770000


In [6]:
# Data subset for tuning
new_data = data
#new_data = data.iloc[:250,]

In [7]:
# Convert data to numpy array
dataset = new_data.values

# Get training set size 80:10:10 split
training_data_len = math.ceil(len(dataset) * 0.8)
testing_data_len = math.ceil((len(dataset) - training_data_len)*0.5)
val_data_len = len(dataset) - training_data_len - testing_data_len
print('Training data length: ' + str(training_data_len))
print('Testing data length: ' + str(testing_data_len))
print('Validation data length: ' + str(val_data_len))

Training data length: 1994
Testing data length: 249
Validation data length: 249


In [8]:
# Scaler for data
scaler = MinMaxScaler(feature_range=(0,1))
# scaled_data = scaler.fit_transform(dataset)

In [9]:
# Create training dataset
train_data = dataset[0:training_data_len,:]

# Split to x_train and y_train
x_train = train_data[:,:51]
y_train = train_data[:,51]

# Create scaled test dataset
test_and_val_data = dataset[training_data_len:,:]

# Split to x_val and y_val
x_val = test_and_val_data[0:testing_data_len,:51]
y_val = test_and_val_data[0:testing_data_len,51]

# Split to x_test and y_test
x_test = test_and_val_data[testing_data_len:,:51]
y_test = test_and_val_data[testing_data_len:,51]

# Show shapes of sets
print('x_train shape: ' + str(x_train.shape))
print('y_train shape: ' + str(y_train.shape))
print()
print('x_val shape: ' + str(x_val.shape))
print('y_val shape: ' + str(y_val.shape))
print()
print('x_test shape: ' + str(x_test.shape))
print('y_test shape: ' + str(y_test.shape))

x_train shape: (1994, 51)
y_train shape: (1994,)

x_val shape: (249, 51)
y_val shape: (249,)

x_test shape: (249, 51)
y_test shape: (249,)


In [10]:
x_train

array([[1.64217858e+01, 1.65357143e+01, 1.62707143e+01, ...,
        1.65080979e-01, 1.31248148e-01, 1.63132120e+01],
       [1.63560715e+01, 1.64182142e+01, 1.60885715e+01, ...,
        1.59885284e-01, 1.46785333e-01, 1.61913601e+01],
       [1.61253573e+01, 1.63535715e+01, 1.61250000e+01, ...,
        1.40642637e-01, 6.38885060e-02, 1.62310985e+01],
       ...,
       [1.25830000e+02, 1.27920000e+02, 1.25140000e+02, ...,
        1.85553870e-02, 1.99921429e-01, 1.27739414e+02],
       [1.28950000e+02, 1.30390000e+02, 1.28520000e+02, ...,
        4.48861810e-02, 1.25479412e-01, 1.30160056e+02],
       [1.29800000e+02, 1.33040000e+02, 1.29470000e+02, ...,
        1.03418236e-01, 2.73479487e-01, 1.32805757e+02]])

In [11]:
# Splits preview
print(y_train)
print(y_val)
print(y_test)

[ 16.31321425  16.1775      16.24071425 ... 127.9        130.36
 132.995     ]
[131.24 134.43 132.03 134.5  134.16 134.84 133.11 133.5  131.94 134.32
 134.72 134.39 133.58 133.48 131.46 132.54 127.85 128.1  129.74 130.21
 126.85 125.91 122.77 124.97 127.45 126.27 124.85 124.69 127.31 125.43
 127.1  126.9  126.85 125.28 124.61 124.28 125.06 123.54 125.89 125.9
 126.74 127.13 126.11 127.35 130.48 129.64 130.15 131.79 130.46 132.3
 133.98 133.7  133.41 133.11 134.78 136.33 136.96 137.27 139.96 142.02
 144.57 143.24 145.11 144.5  145.64 149.15 148.48 146.39 142.45 146.15
 145.4  146.8  148.56 148.99 146.77 144.98 145.64 145.86 145.52 147.36
 146.95 147.06 146.14 146.09 145.6  145.86 148.89 149.1  151.12 150.19
 146.36 146.7  148.19 149.71 149.62 148.36 147.54 148.6  153.12 151.83
 152.51 153.65 154.3  156.69 155.11 154.07 148.97 149.55 148.12 149.03
 148.79 146.06 142.94 143.43 145.85 146.83 146.92 145.37 141.91 142.83
 141.5  142.65 139.14 141.11 142.   143.29 142.9  142.81 141.51 140.91


In [12]:
# Convert sets to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

x_val = np.array(x_test)
y_val = np.array(y_test)

x_test = np.array(x_test)
y_test = np.array(y_test)

# Scale data
scaled_data = scaler.fit_transform(dataset)

x_train = scaler.fit_transform(x_train)
y_train = scaler.fit_transform(y_train.reshape(-1,1))

x_val = scaler.fit_transform(x_val)
y_val = scaler.fit_transform(y_val.reshape(-1,1))

x_test = scaler.fit_transform(x_test)
y_test = scaler.fit_transform(y_test.reshape(-1,1))

x_train

array([[0.01885966, 0.01712881, 0.01868894, ..., 0.78375968, 0.4121214 ,
        0.01733483],
       [0.01835269, 0.01623046, 0.01726017, ..., 0.77392832, 0.4330199 ,
        0.01639152],
       [0.01657281, 0.01573623, 0.01754592, ..., 0.73751712, 0.32151841,
        0.01669916],
       ...,
       [0.86290998, 0.86872551, 0.87268769, ..., 0.50650193, 0.50449132,
        0.87993629],
       [0.88697981, 0.88761008, 0.89920129, ..., 0.55632543, 0.40436208,
        0.8986756 ],
       [0.8935373 , 0.90787084, 0.90665333, ..., 0.66708059, 0.60343157,
        0.91915718]])

In [13]:
train_x = x_train
train_y = y_train
val_x = x_val
val_y = y_val
test_x = x_test
test_y = y_test

# Code below modified from: https://github.com/ChickenBenny/Stock-prediction-with-GAN-and-WGAN/blob/main/wgan-gp_demo.ipynb

#Variational Autoencoder
class VAE(nn.Module):
    def __init__(self, config, latent_dim):
        super().__init__()

        modules = []
        for i in range(1, len(config)):
            modules.append(
                nn.Sequential(
                    nn.Linear(config[i - 1], config[i]),
                    nn.ReLU()
                )
            )
        
        self.encoder = nn.Sequential(*modules)
        self.fc_mu = nn.Linear(config[-1], latent_dim)
        self.fc_var = nn.Linear(config[-1], latent_dim)

        modules = []
        self.decoder_input = nn.Linear(latent_dim, config[-1])

        for i in range(len(config) - 1, 1, -1):
            modules.append(
                nn.Sequential(
                    nn.Linear(config[i], config[i - 1]),
                    nn.ReLU()
                )
            )       
        modules.append(
            nn.Sequential(
                nn.Linear(config[1], config[0]),
                nn.Sigmoid()
            )
        ) 

        self.decoder = nn.Sequential(*modules)

    def encode(self, x):
        result = self.encoder(x)
        mu = self.fc_mu(result)
        logVar = self.fc_var(result)
        return mu, logVar

    def decode(self, x):
        result = self.decoder(x)
        return result

    def reparameterize(self, mu, logVar):
        std = torch.exp(0.5* logVar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, x):
        mu, logVar = self.encode(x)
        z = self.reparameterize(mu, logVar)
        output = self.decode(z)
        return output, z, mu, logVar

train_loader = DataLoader(TensorDataset(torch.from_numpy(train_x).float()), batch_size = 128, shuffle = False)
model = VAE([6, 64, 64, 64, 5], 5)

use_cuda = 1
device = torch.device("cuda" if (torch.cuda.is_available() & use_cuda) else "cpu")
num_epochs = 180
learning_rate = 0.0001
model = model.to(device)   
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

hist = np.zeros(num_epochs) 
for epoch in range(num_epochs):
    total_loss = 0
    loss_ = []
    for (x, ) in train_loader:
        x = x.to(device)
        output, z, mu, logVar = model(x)
        kl_divergence = 0.5* torch.sum(-1 - logVar + mu.pow(2) + logVar.exp())
        loss = F.binary_cross_entropy(output, x) + kl_divergence
        loss.backward()
        optimizer.step()
        loss_.append(loss.item())
    hist[epoch] = sum(loss_)
    print('[{}/{}] Loss:'.format(epoch+1, num_epochs), sum(loss_))

plt.figure(figsize=(12, 6))
plt.plot(hist)

model.eval()
_, VAE_train_x, train_x_mu, train_x_var = model(torch.from_numpy(train_x).float().to(device))
_, VAE_test_x, test_x_mu, test_x_var = model(torch.from_numpy(test_x).float().to(device))

In [14]:
def sliding_window(x, y, window):
    x_ = []
    y_ = []
    y_gan = []
    for i in range(window, x.shape[0]):
        tmp_x = x[i - window: i, :]
        tmp_y = y[i]
        tmp_y_gan = y[i - window: i + 1]
        x_.append(tmp_x)
        y_.append(tmp_y)
        y_gan.append(tmp_y_gan)
    x_ = torch.from_numpy(np.array(x_)).float()
    y_ = torch.from_numpy(np.array(y_)).float()
    y_gan = torch.from_numpy(np.array(y_gan)).float()
    return x_, y_, y_gan

# For VAE
# train_x = np.concatenate((train_x, VAE_train_x.cpu().detach().numpy()), axis = 1)
# test_x = np.concatenate((test_x, VAE_test_x.cpu().detach().numpy()), axis = 1)

train_x_slide, train_y_slide, train_y_gan = sliding_window(train_x, train_y, 3)
val_x_slide, val_y_slide, val_y_gan = sliding_window(val_x, val_y, 3)
test_x_slide, test_y_slide, test_y_gan = sliding_window(test_x, test_y, 3)
print(f'train_x: {train_x_slide.shape} train_y: {train_y_slide.shape} train_y_gan: {train_y_gan.shape}')
print(f'val_x: {val_x_slide.shape} val_y: {val_y_slide.shape} val_y_gan: {val_y_gan.shape}')
print(f'test_x: {test_x_slide.shape} test_y: {test_y_slide.shape} test_y_gan: {test_y_gan.shape}')


train_x: torch.Size([1991, 3, 51]) train_y: torch.Size([1991, 1]) train_y_gan: torch.Size([1991, 4, 1])
val_x: torch.Size([246, 3, 51]) val_y: torch.Size([246, 1]) val_y_gan: torch.Size([246, 4, 1])
test_x: torch.Size([246, 3, 51]) test_y: torch.Size([246, 1]) test_y_gan: torch.Size([246, 4, 1])


In [15]:
# Build Generator: Long Short Term Memory Network
class Generator(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        
        #Build LSTM
        self.LSTM = nn.LSTM(input_size=input_size, hidden_size=512, num_layers=1, batch_first = True)
        self.output_layer = nn.Linear(512, 1, bias=False)
        
#         self.gru_1 = nn.GRU(input_size, 1024, batch_first = True)
#         self.gru_2 = nn.GRU(1024, 512, batch_first = True)
#         self.gru_3 = nn.GRU(512, 256, batch_first = True)
#         self.linear_1 = nn.Linear(256, 128)
#         self.linear_2 = nn.Linear(128, 64)
#         self.linear_3 = nn.Linear(64, 1)
#         self.dropout = nn.Dropout(0.2)
        

    def forward(self, x):
        # initialize hidden state
        h0 = torch.zeros(1, x.size(0), 512).requires_grad_()
        
        # initialize cell state
        c0 = torch.zeros(1, x.size(0), 512).requires_grad_()
        
        # Data Path through NN
        LSTM_out, (LSTM_hn, LSTM_cn) = self.LSTM(x, (h0.detach(), c0.detach()))
        output = self.output_layer(LSTM_out[:, -1, :])
        return output
        
        
        
#         use_cuda = 1
#         device = torch.device("cuda" if (torch.cuda.is_available() & use_cuda) else "cpu")
#         h0 = torch.zeros(1, x.size(0), 1024).to(device)
#         out_1, _ = self.gru_1(x, h0)
#         out_1 = self.dropout(out_1)
#         h1 = torch.zeros(1, x.size(0), 512).to(device)
#         out_2, _ = self.gru_2(out_1, h1)
#         out_2 = self.dropout(out_2)
#         h2 = torch.zeros(1, x.size(0), 256).to(device)
#         out_3, _ = self.gru_3(out_2, h2)
#         out_3 = self.dropout(out_3)
#         out_4 = self.linear_1(out_3[:, -1, :])
#         out_5 = self.linear_2(out_4)
#         out_6 = self.linear_3(out_5)
#         return out_6

# Build Discriminator: Convolutional Neural Network Version 3.2
class Discriminator(nn.Module):       
    def __init__(self):
        super().__init__()
        
        # 1-Dimensional Convolutional Layers with LeakyReLU and Batch Normalization
        self.conv_layer0 = nn.Conv1d(4, 32, kernel_size=5, stride=1, padding='same')
        self.conv_layer1 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding='same')
        self.leakyrelu_layer2 = nn.LeakyReLU(0.01)
        self.conv_layer3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding='same')
        self.leakyrelu_layer4 = nn.LeakyReLU(0.01)
#         self.batchnorm_layer5 = nn.BatchNorm1d(128)
        self.conv_layer6 = nn.Conv1d(128, 256, kernel_size=5, stride=1, padding='same')
        self.leakyrelu_layer7 = nn.LeakyReLU(0.01)
#         self.batchnorm_layer8 = nn.BatchNorm1d(256)
        
        # Fully Connected Layers
        self.dense_layer9 = nn.Linear(256, 256, bias=False)
#         self.batchnorm_layer10 = nn.BatchNorm1d(256)
        self.leakyrelu_layer11 = nn.LeakyReLU(0.01)
        self.dense_layer12 = nn.Linear(256, 256, bias=False)
        self.activation_layer13 = nn.ReLU()
        self.dense_layer14 = nn.Linear(256, 1)
        

    def forward(self, x):
        conv_layer0 = self.conv_layer0(x)
        conv_layer1 = self.conv_layer1(conv_layer0)
        leakyrelu_layer2 = self.leakyrelu_layer2(conv_layer1)
        conv_layer3 = self.conv_layer3(leakyrelu_layer2)
        leakyrelu_layer4 = self.leakyrelu_layer4(conv_layer3)
#         batchnorm_layer5 = self.batchnorm_layer5(leakyrelu_layer4)
        conv_layer6 = self.conv_layer6(leakyrelu_layer4)
        leakyrelu_layer7 = self.leakyrelu_layer7(conv_layer6)
#         batchnorm_layer8 = self.batchnorm_layer8(leakyrelu_layer7)
        reshape =  leakyrelu_layer7.reshape(leakyrelu_layer7.shape[0], leakyrelu_layer7.shape[1])
        dense_layer9 = self.dense_layer9(reshape)
#         batchnorm_layer10 = self.batchnorm_layer10(dense_layer9)
        leakyrelu_layer11 = self.leakyrelu_layer11(dense_layer9)
        dense_layer12 = self.dense_layer12(leakyrelu_layer11)
        activation_layer13 = self.activation_layer13(dense_layer12)
        dense_layer14 = self.dense_layer14(activation_layer13)
        return dense_layer14

In [16]:
# Gradient Penalty Implementation
# Modified From: https://github.com/Zeleni9/pytorch-wgan/blob/master/models/wgan_gradient_penalty.py
def calculate_gradient_penalty(critic, real_data, fake_data):
    eta = torch.FloatTensor(fake_data.size(0),1,1).uniform_(0,1)
    eta = eta.expand(fake_data.size(0), real_data.size(1), real_data.size(2))

#     print(eta.size())
#     print(fake_data.size())
    interpolated = eta * real_data + ((1 - eta) * fake_data)

    # define it to calculate gradient
    interpolated = Variable(interpolated, requires_grad=True)

    # calculate probability of interpolated examples
    prob_interpolated = critic(interpolated)

    # calculate gradients of probabilities with respect to examples
    gradients = torch_grad(outputs=prob_interpolated, inputs=interpolated,
                           grad_outputs=torch.ones(
                               prob_interpolated.size()),
                           create_graph=True, retain_graph=True)[0]

    grad_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * 10
    return grad_penalty

In [ ]:
use_cuda = 1
device = torch.device("cuda" if (torch.cuda.is_available() & use_cuda) else "cpu")

from sklearn.model_selection import ParameterGrid

param_grid = {'epochs':[100, 125, 150, 175, 200, 250],
              'learning_rate': [0.0001, 0.00001],
              'batch_size' : [32, 64, 128],
              'critic_iterations' :[1,2,3,4,5]}

grid = ParameterGrid(param_grid)

full_params_list = list()
iteration_count = 0

for params in grid:
    iteration_count+=1
    print(f'---------- Starting training with new params [{iteration_count}/{len(grid)}] ----------')
    print('')
    print(params)
    
    # Hyperparameters set by grid search
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']
    num_epochs = params['epochs']
    critic_iterations = params['critic_iterations']
    
    # Data Loader    
    trainDataloader = DataLoader(TensorDataset(train_x_slide, train_y_gan), batch_size = batch_size, shuffle = False)

    # Build Models
    modelG = Generator(51).to(device)
    modelD = Discriminator().to(device)

    # Build Optimizers
    optimizerG = torch.optim.RMSprop(modelG.parameters(), lr=learning_rate, alpha = 0.9)
    optimizerD = torch.optim.RMSprop(modelD.parameters(), lr=learning_rate, alpha = 0.9)

    # optimizerG = torch.optim.Adam(modelG.parameters(), lr = learning_rate, betas = (0.0, 0.9), weight_decay = 1e-3)
    # optimizerD = torch.optim.Adam(modelD.parameters(), lr = learning_rate, betas = (0.0, 0.9), weight_decay = 1e-3)

    # History Storage
    histG = np.zeros(num_epochs)
    histD = np.zeros(num_epochs)
    
    for epoch in range(num_epochs):
        loss_G = []
        loss_D = []
        for (x, y) in trainDataloader:
            x = x.to(device)
            y = y.to(device)

            lossD_arr = []
            costG = 0

            # Generate Fake Data
            fake_data = modelG(x)
            fake_data = torch.cat([fake_data.reshape(-1, 1, 1), y[:, :3, :]], axis = 1)

            for i in range(critic_iterations):
                # Train Discriminator
                critic_loss_real = 0
                critic_loss_fake = 0
                critic_wasserstein = 0


                    # Train on real closing prices
                real_data = y
                critic_loss_real = modelD(real_data)
                critic_loss_real = torch.mean(critic_loss_real)

                    # Train on fake closing prices      
                critic_loss_fake = modelD(fake_data)
                critic_loss_fake = torch.mean(critic_loss_fake)


                    # Calculate Gradient Penalty
                gradient_penalty = calculate_gradient_penalty(modelD, real_data, fake_data)

                    # Compute Loss
                lossD = critic_loss_fake - critic_loss_real + gradient_penalty

                    # Zero Gradients
                modelD.zero_grad()

                    # Back Propogation
                lossD.backward(retain_graph = True)    
                lossD_arr.append(lossD)
                critic_wasserstein = -(critic_loss_real - critic_loss_fake)

                # Optimizer step
                optimizerD.step()

            # Train Generator

                # Compute Loss
            lossG = modelD(fake_data)
            lossG = -torch.mean(lossG)
            modelG.zero_grad()
            lossG.backward()
            optimizerG.step()

            # Append Losses
            loss_D.append((torch.mean(torch.stack(lossD_arr))).item())
            loss_G.append(lossG.item())

        # Print Losses
        histG[epoch] = sum(loss_G) 
        histD[epoch] = sum(loss_D)    
#         print(f'[{epoch+1}/{num_epochs}] LossD: {sum(loss_D)} LossG:{sum(loss_G)}')
        
    #Evaluate results for current params
    modelG.eval()
    pred_y_train = modelG(train_x_slide.to(device))
    pred_y_val = modelG(val_x_slide.to(device))

    y_train_true = scaler.inverse_transform(train_y_slide)
    y_train_pred = scaler.inverse_transform(pred_y_train.cpu().detach().numpy())

    y_val_true = scaler.inverse_transform(val_y_slide)
    y_val_pred = scaler.inverse_transform(pred_y_val.cpu().detach().numpy())

    train_RMSE = math.sqrt(mean_squared_error(y_train_true, y_train_pred))
    val_RMSE = math.sqrt(mean_squared_error(y_val_true, y_val_pred))

    #Store RMSE values for min search later
    params['train_RMSE'] = train_RMSE
    params['val_RMSE'] = val_RMSE    
    print('train_RMSE, val_RMSE = {x:.3f},{y:.3f}'.format(x = train_RMSE, y = val_RMSE))
    full_params_list.append(params)
    print('---------- Ending training with new params ----------')
    print()

---------- Starting training with new params [1/180] ----------

{'batch_size': 32, 'critic_iterations': 1, 'epochs': 100, 'learning_rate': 0.0001}


In [ ]:
file = open('params.txt','w')
for param in sorted_params:
    print(param)
    file.write(str(param)+"\n")
file.close()

In [ ]:
plt.figure(figsize = (6, 4))
plt.plot(histD, color = 'black', label = 'Discriminator Loss')
plt.title('Discriminator Loss with Gradient Penalty')
plt.xlabel('Epochs')

In [ ]:
plt.figure(figsize = (6, 4))
plt.plot(histG, color = 'blue', label = 'Generator Loss')
plt.title('Generator Loss')
plt.xlabel('Epochs')

In [ ]:
modelG.eval()
pred_y_train = modelG(train_x_slide.to(device))
pred_y_test = modelG(test_x_slide.to(device))

y_train_true = scaler.inverse_transform(train_y_slide)
y_train_pred = scaler.inverse_transform(pred_y_train.cpu().detach().numpy())

y_test_true = scaler.inverse_transform(test_y_slide)
y_test_pred = scaler.inverse_transform(pred_y_test.cpu().detach().numpy())

In [ ]:

plt.figure(figsize=(6, 4))
plt.plot(y_train_true, color = 'black', label = 'Acutal Price')
plt.plot(y_train_pred, color = 'blue', label = 'Predict Price')
plt.title('WGAN-GP Prediction over the Training Dataset')
plt.ylabel('$AAPL Stock Price')
plt.xlabel('Days')
plt.legend(loc = 'upper right')

MSE = mean_squared_error(y_train_true, y_train_pred)
RMSE = math.sqrt(MSE)
print(f'Training dataset RMSE:{RMSE}')

In [ ]:

plt.figure(figsize=(6, 4))
plt.plot(y_test_true, color = 'black', label = 'Acutal Price')
plt.plot(y_test_pred, color = 'blue', label = 'Predict Price')
plt.title('WGAN-GP Prediction over the Testing Dataset')
plt.ylabel('$AAPL Stock Price')
plt.xlabel('Days')
plt.legend(loc = 'upper right')

MSE = mean_squared_error(y_test_true, y_test_pred)
RMSE = math.sqrt(MSE)
print(f'Testing dataset RMSE:{RMSE}')

In [ ]:
from sklearn.model_selection import ParameterGrid
param_grid = {'epochs':[100, 150, 200, 250, 300],
              'learning_rate': [0.1, 0.01, 0.001, 0.0001, 0.000001],
              'batch_size' : [16,32,64,128,256],
              'critic_iterations' :[1,5,10,25,50,75,100]}

grid = ParameterGrid(param_grid)

full_params_list = list()

for params in grid:
    full_params_list.append(params)
    
maxLR = max(full_params_list, key=lambda x:x['learning_rate'])
print(maxLR)